In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv 

options = webdriver.ChromeOptions()
options.add_argument("--lang=en")

driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))

# Create lists to store data
results = []

for page_number in range(1, 21):
    web = f'https://www.amazon.com/s?k=pc&crid=13K6ZIW5WECV6&sprefix=pc%2Caps%2C288&ref=nb_sb_noss_1&page={page_number}'
    driver.get(web)

    # Wait for search results to load
    items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16")]')))

    for item in items:
        name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]').text

        whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
        fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')
        if whole_price and fraction_price:
            price = '.'.join([whole_price[0].text, fraction_price[0].text])
        else:
            price = 'N/A'

        link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute('href')

        # Store data in a dictionary
        result = {
            'PC Name': name,
            'Price': price,
            'Link': link
        }

        # Append the dictionary to the results list
        results.append(result)

    driver.quit()

with open('AmazonPcDataset.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    header = ['PC Name', 'Price', 'Link']
    writer.writerow(header)
    for result in results:
        writer.writerow([result['PC Name'], result['Price'], result['Link']])


KeyboardInterrupt: 